In [2]:
import psycopg2
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

host="13.43.84.151"
dbname="postgres"
user="ping-fan.chen.23@ucl.ac.uk"
password="khsLcR"
port="5432"
schema="schema_pingfanchen23uclacuk"

In [9]:
conn = psycopg2.connect (
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)
print("Database connection established")

Database connection established


In [5]:
# fetch the java driver for postgress to allow Spark to connect to postgress
!curl -o postgresql-42.3.2.jar https://jdbc.postgresql.org/download/postgresql-42.3.2.jar
# uses the curl command to download the PostgreSQL JDBC driver from the official site.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1015k  100 1015k    0     0   642k      0  0:00:01  0:00:01 --:--:--  642k


In [11]:
import os
current_dir = os.getcwd() # assigns the current working directory to the variable current_dir
print(current_dir)

jar_location = current_dir + "/postgresql-42.3.2.jar"
# Constructs the file path for the PostgreSQL JDBC driver JAR file by appending the filename to the current directory path
jar_location

/home/jovyan/dataengineer


'/home/jovyan/dataengineer/postgresql-42.3.2.jar'

In [37]:
from pyspark.sql import SparkSession
# Initialize Spark Session
spark = SparkSession.builder \
    .config("spark.driver.extraClassPath", jar_location)\
    .appName("Enhanced Data Warehouse ETL") \
    .getOrCreate()
# Sets the configuration property spark.driver.extraClassPath to the path of the JDBC driver.
# This tells Spark where to find the driver for the database connection.

24/03/08 17:26:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### ZIP Code

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, lit, lead

# using api
import requests

states = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA',
        'KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM',
        'NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV',
        'WI','WY']
city = ""
states_zip = {}
for state in states:
    api_url = 'https://api.api-ninjas.com/v1/zipcode?city={}&state={}'.format(city, state)

    response = requests.get(api_url+city, headers={'X-Api-Key': '0+WwzqaBAr5kWjywXArIpA==DlpdDtBfCV9Pb7Nw'})
    if response.status_code == requests.codes.ok:
        data = response.json()
        df = spark.createDataFrame(data)
        states_zip[state] = df
    else:
        print("Error:", response.status_code, response.text)

from functools import reduce
from pyspark.sql import DataFrame
df_all = reduce(DataFrame.unionAll, states_zip.values())
# Show the dataframe
df_all.show()

In [12]:
df_all.show()

+----------+--------------+-------+-----------------+-------+--------+-----+---------------+-----+--------+
|area_codes|          city|country|           county|    lat|     lon|state|       timezone|valid|zip_code|
+----------+--------------+-------+-----------------+-------+--------+-----+---------------+-----+--------+
|     [205]|         Moody|     US| St. Clair County|33.6034|-86.4944|   AL|America/Chicago| true|   35004|
|     [205]|    Adamsville|     US| Jefferson County|33.5929|-86.9940|   AL|America/Chicago| true|   35005|
|     [205]|         Adger|     US| Jefferson County|33.4462|-87.2230|   AL|America/Chicago| true|   35006|
|     [205]|     Alabaster|     US|    Shelby County|33.2187|-86.7835|   AL|America/Chicago| true|   35007|
|     [256]|Alexander City|     US|Tallapoosa County|32.9011|-85.9178|   AL|America/Chicago| true|   35010|
|     [256]|Alexander City|     US|Tallapoosa County|32.9394|-85.9466|   AL|America/Chicago| true|   35011|
|     [205]|       Allgood| 

In [32]:
from sqlalchemy import create_engine
# JDBC URL
url = f"jdbc:postgresql://{host}:{port}/{dbname}?currentSchema=schema_pingfanchen23uclacuk"

# Current directory and JDBC driver location
current_dir = os.getcwd()
jar_location = os.path.join(current_dir, "postgresql-42.3.2.jar")

properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

df_all.write.jdbc(url=url, table="zip_code_by_city", mode="overwrite", properties=properties)

print("Data has been successfully stored in the database")

Data has been successfully stored in the database


### sales tax

In [ ]:
import psycopg2

connection_params = {
    "host":"13.43.84.151",
    "user": "ping-fan.chen.23@ucl.ac.uk",
    "password": "khsLcR",
    "dbname": "postgres",
}

conn = psycopg2.connect(**connection_params)

cur = conn.cursor()
cur = conn.cursor()
cur.execute("SELECT zip_code FROM schema_pingfanchen23uclacuk.zip_code_by_city;")
df = cur.fetchall()
zip_list = [i[0] for i in df]
print(zip_list[:10])
cur.close()

In [ ]:
import requests
df_list = []
for zip_code in zip_list[::100]:
    api_url = 'https://api.api-ninjas.com/v1/salestax?zip_code={}'.format(zip_code)
    response = requests.get(api_url, headers={'X-Api-Key': '0+WwzqaBAr5kWjywXArIpA==DlpdDtBfCV9Pb7Nw'})
    if response.status_code == requests.codes.ok:
        data = response.json()
        df = spark.createDataFrame(data)
        df_list.append(df)
else:
    print("Error:", response.status_code, response.text)

from functools import reduce
from pyspark.sql import DataFrame
df_all = reduce(DataFrame.unionAll, df_list)
# Show the dataframe
df_all.show()

# JDBC URL
url = f"jdbc:postgresql://{host}:{port}/{dbname}?currentSchema=schema_pingfanchen23uclacuk"
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}
df_all.write.jdbc(url=url, table="sale_tax_by_zip", mode="overwrite", properties=properties)

print("Data has been successfully stored in the database")

### Median Price

In [ ]:
def reshape_table(column,df):
    table = df[column].reset_index()
    city,state = column[0].split(',')
    table['State'] = state
    table['City']=city
    table['Price'] = table[column]
    table['Time'] = table['index']
    table = table[['State','City','Time','Price']]
    return table

In [ ]:
import pandas as pd
df_median = pd.read_csv('median_rental_price.csv',skiprows=0)
df_median = df_median.loc[1:,df_median.columns[~df_median.columns.isin(['RegionID','SizeRank','RegionType'])]]
# df_median.groupby('StateName', group_keys=True)[df_median.columns].apply(lambda x: x)
df = pd.DataFrame(df_median[df_median.columns[2:]].values.T,index = df_median.columns[2:],columns= [df_median['RegionName'].values])
df.reset_index(names='Time').head()
lst = []
for column in df.columns[1:]:
    lst.append(reshape_table(column,df))

In [ ]:
# df = pd.concat(lst)
# df.columns = ['State','City','Time','Price']
df['State'].astype('str')
df['City'].astype('str')
df.dropna(subset = ['Price'],inplace=True)

In [ ]:
df.info()
df.to_csv('median_rent_price.csv',index=False)

In [21]:
median_price = spark.read.option("header", "true").csv('median_rent_price.csv')
median_price.show(5)

24/03/08 17:05:19 INFO InMemoryFileIndex: It took 91 ms to list leaf files for 1 paths.
24/03/08 17:05:19 INFO InMemoryFileIndex: It took 2 ms to list leaf files for 1 paths.
24/03/08 17:05:22 INFO FileSourceStrategy: Pushed Filters: 
24/03/08 17:05:22 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#0, None)) > 0)
24/03/08 17:05:23 INFO CodeGenerator: Code generated in 400.774784 ms
24/03/08 17:05:23 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 200.0 KiB, free 413.7 MiB)
24/03/08 17:05:23 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 413.7 MiB)
24/03/08 17:05:23 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on jupyter-ping-2dfan-2echen-2e23-40ucl-2eac-2euk:44103 (size: 34.5 KiB, free: 413.9 MiB)
24/03/08 17:05:23 INFO SparkContext: Created broadcast 0 from csv at NativeMethodAccessorImpl.java:0
24/03/08 17:05:23 INFO FileSourceScanExec: Planning scan with bin packing, max 

+-----+-----------+----------+--------+
|State|       City|      Time|   Price|
+-----+-----------+----------+--------+
|   CA|Los Angeles|2008-02-29|470000.0|
|   CA|Los Angeles|2008-03-31|456000.0|
|   CA|Los Angeles|2008-04-30|457000.0|
|   CA|Los Angeles|2008-05-31|440000.0|
|   CA|Los Angeles|2008-06-30|435000.0|
+-----+-----------+----------+--------+
only showing top 5 rows



24/03/08 17:05:25 INFO Executor: Finished task 0.0 in stage 1.0 (TID 1). 1645 bytes result sent to driver
24/03/08 17:05:25 INFO TaskSetManager: Finished task 0.0 in stage 1.0 (TID 1) in 191 ms on jupyter-ping-2dfan-2echen-2e23-40ucl-2eac-2euk (executor driver) (1/1)
24/03/08 17:05:25 INFO TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
24/03/08 17:05:25 INFO DAGScheduler: ResultStage 1 (showString at NativeMethodAccessorImpl.java:0) finished in 0.204 s
24/03/08 17:05:25 INFO DAGScheduler: Job 1 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/08 17:05:25 INFO TaskSchedulerImpl: Killing all running tasks in stage 1: Stage finished
24/03/08 17:05:25 INFO DAGScheduler: Job 1 finished: showString at NativeMethodAccessorImpl.java:0, took 0.209076 s
24/03/08 17:05:25 INFO CodeGenerator: Code generated in 11.493138 ms
24/03/08 17:08:21 INFO BlockManagerInfo: Removed broadcast_4_piece0 on jupyter-ping-2dfan-2echen-2e23-40ucl-2

In [ ]:
# JDBC URL
url = f"jdbc:postgresql://{host}:{port}/{dbname}?currentSchema=schema_pingfanchen23uclacuk"

# Current directory and JDBC driver location
current_dir = os.getcwd()
jar_location = os.path.join(current_dir, "postgresql-42.3.2.jar")

properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

median_price.write.jdbc(url=url, table="median_price", mode="overwrite", properties=properties)

print("Data has been successfully stored in the database")

### Check data is in the right format of database

In [11]:
# Create a cursor object
cur = conn.cursor()

# Query to get all schemas
cur.execute("SELECT schema_name FROM information_schema.schemata;")
schemas = cur.fetchall()

# Print the available schemas
print("Available schemas:")
for schema in schemas:
    print(schema[0])

Available schemas:
pg_catalog
information_schema
schema_pingfanchen23uclacuk
public


In [12]:
# Query to get all table names in a specific schema
cur = conn.cursor()
schema_to_check = 'schema_pingfanchen23uclacuk'  # replace with your actual schema name
cur.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema_to_check}';")
tables = cur.fetchall()

# Print the tables in the schema
print(f"Tables in schema '{schema_to_check}':")
for table in tables:
    print(table[0])

# Close the cursor and the connection
cur.close()
conn.close()

Tables in schema 'schema_pingfanchen23uclacuk':
zip_code_by_city
sale_tax_by_zip
median_price


In [20]:
import pandas as pd
from pyspark.sql import SparkSession
conn = psycopg2.connect (
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)
cur = conn.cursor()
# Perform the SQL join query using state as the key, combining the upper,median,lower quartile values together 

schema = 'schema_pingfanchen23uclacuk'
# Execute the query
cur.execute(f"""
SELECT table_name FROM information_schema.tables
WHERE table_schema = '{schema}'""")

tables = cur.fetchall()
for table in tables:
    table_name = table[0]
    print(f"Data from table: {table_name}")
    
    # Fetch data from each table
    query = f"SELECT * FROM {schema}.{table_name};"
    df = pd.read_sql_query(query, conn)
    
    # Display the DataFrame
    print(df.head())
cur.close()
conn.close()
# Write the cleaned quartile value DataFrame to PostgreSQL
# df.write.jdbc(url=url, table="combined_value_quartile", mode="overwrite", properties=properties)

Data from table: zip_code_by_city


/tmp/ipykernel_75/3626738777.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


  area_codes            city country             county      lat       lon  \
0      [205]           Moody      US   St. Clair County  33.6034  -86.4944   
1      [205]      Adamsville      US   Jefferson County  33.5929  -86.9940   
2      [205]           Adger      US   Jefferson County  33.4462  -87.2230   
3      [205]       Alabaster      US      Shelby County  33.2187  -86.7835   
4      [256]  Alexander City      US  Tallapoosa County  32.9011  -85.9178   

  state         timezone  valid zip_code  
0    AL  America/Chicago   True    35004  
1    AL  America/Chicago   True    35005  
2    AL  America/Chicago   True    35006  
3    AL  America/Chicago   True    35007  
4    AL  America/Chicago   True    35010  
Data from table: sale_tax_by_zip
  additional_rate city_rate county_rate state_rate total_rate zip_code
0        0.020000  0.040000    0.000000   0.040000   0.100000    35004
1        0.020000  0.040000    0.000000   0.040000   0.100000    35173
2        0.000000  0.030000

/tmp/ipykernel_75/3626738777.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_75/3626738777.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
